In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import cv2
#from PIL import Image
import torch
import os
from matplotlib import pyplot as plt
import torchvision
import torchvision.transforms as transforms
from skimage import io
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%matplotlib inline
plt.ion() 

In [2]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
filepaths = []
grp_dataset = {'cats':[], 'dogs':[]}
grp_dataset_nobkg = {'cats':[], 'dogs':[]}
dir_cats = 'cat-and-dog/training_set/cats'
dir_dogs = 'cat-and-dog/dogs'

#test_plane = '/kaggle/input/machine-plane/plane_PNG5238.png'
#test_cat = '/kaggle/input/catsnobackground/5a5a8b8b14d8c4188e0b08e7.png'
#test_dog = '/kaggle/input/dogsnobackground/dog-png-transparent-background-basset-hound-png-file-324.png'

for dirname, _, filenames in os.walk('cat-and-dog'):
    for filename in filenames:
        filepath = os.path.join(dirname, filename)
        aux = filepath.split('/')
        if 'dogs' in aux:
            grp_dataset['dogs'].append(filepath)
        elif 'cats' in aux:
            grp_dataset['cats'].append(filepath)
#        if 'catsnobackground' in filepath:
#            grp_dataset_nobkg['cats'].append(filepath)
#        if 'dogsnobackground' in filepath:
#            grp_dataset_nobkg['dogs'].append(filepath)

# Any results you write to the current directory are saved as output.
print(len(grp_dataset['cats']))
print(len(grp_dataset['dogs']))
print(grp_dataset['cats'][0])
print(grp_dataset['dogs'][0])
#print(grp_dataset_nobkg['cats'][0])
#print(grp_dataset_nobkg['dogs'][0])


5012
5019
cat-and-dog/test_set/cats/cat.4001.jpg
cat-and-dog/dogs/dog.1.jpg


In [23]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(165888, 4096)
        self.fc2 = nn.Linear(4096, 2048)
        self.fc3 = nn.Linear(2048, 1024)
        self.drop30 = torch.nn.Dropout(p=0.3)
        self.drop70 = torch.nn.Dropout(p=0.7)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 165888)
        x = self.drop30(x)
        x = F.relu(self.fc1(x))
        x = self.drop30(x)
        x = F.relu(self.fc2(x))
        x = self.drop70(x)
        x = torch.sigmoid(self.fc3(x))
        return x


net = Net()

import torch.optim as optim

#criterion = nn.CrossEntropyLoss()
#criterion = nn.MultiLabelMarginLoss()
criterion = nn.MultiLabelSoftMarginLoss()
#criterion = nn.HingeEmbeddingLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.0001)


In [4]:
import random
random.seed(1234)
if 'rand_idxs' not in locals():
    rand_idxs = []
    for x in range(20):
      rand_idxs.append(random.randint(0,1023))

print(len(rand_idxs))
output_zero = torch.zeros((1024), dtype=torch.long)
output_cat = torch.zeros((1024), dtype=torch.long)
print(output_cat.shape)
for i in rand_idxs:
    output_cat[i] = 1
print(torch.dot(output_cat.T, output_cat) == torch.LongTensor([20]))
#print(torch.dot(output_cat.T, output_cat) == torch.FloatTensor([20]))
output_zero = output_zero.reshape(1,1024)
output_cat = output_cat.reshape(1,1024)
print(output_zero.shape)
print(output_cat.shape)


20
torch.Size([1024])
tensor([True])
torch.Size([1, 1024])
torch.Size([1, 1024])


In [24]:
size_img = 200
size_crop = 100
#transform = transforms.Compose(
#    [transforms.ToTensor(),
#     transforms.Pad(20, padding_mode='reflect'),
#     transforms.Resize((size_img,size_img)),
#     transforms.CenterCrop(size_crop),
#     transforms.ToTensor(), 
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

def load_img(path):
    return io.imread(path,mode)

def sample_rand(ch, sizeH, sizeW):
     return torch.rand((ch, sizeH, sizeW)).unsqueeze(0)
    
def load_imgs(paths):
    for path in paths:
        yield load_img(path)
    
def load_data_img(paths):
    for path in paths:
        yield transform(load_img(path)).unsqueeze(0)


In [25]:
def cat_list(lst_imgs):
    trains = torch.cat((lst_imgs[0], lst_imgs[1]), 0)
    for i in range(2,len(lst_imgs)):
        trains = torch.cat((trains, lst_imgs[i]), 0)
    return trains
    

def create_minibatch(lst_imgs=None, ch=3, sH=200, sW=200, label=None, lbRand=None, presetted=None):
    if presetted is not None:
        trains = cat_list(lst_imgs + presetted)
    else:
        rnd_lst = []
        for i, _ in enumerate(lbRand):
            rnd_lst = sample_rand(ch, sH, sW)
        trains = cat_list(lst_imgs + rnd_lst)
    labels = cat_list(label + lbRand)
    return trains, labels


#def load_data(lst_files, size_batch):
#    for i, _ in range(round(len(lst_files/size_batch))+1):
#        lst_files = []
#        for load_data_img(paths):
    
def save_rnds(rnds, path='cat-and-dog/training_set/rnds'):
    for i, rnd in enumerate(rnds):
        img_rnd = rnd[0]
        sps_rnd = img_rnd.shape
        filename = os.path.join(path, 'rnd.' + str(i) + '.jpg')
        for ch in range(sps_rnd[0]):
            img_rnd[ch] = (img_rnd[ch]*254).int()
        io.imsave(filename, img_rnd.reshape(sps_rnd[2], sps_rnd[1], sps_rnd[0]).numpy().astype(np.uint8))
 

In [7]:
labels_cat = [output_cat] * len(grp_dataset['cats'])
labels_rnd = [output_zero] * int(len(grp_dataset['cats'])*0.7)

print(len(labels_cat))
print(len(labels_rnd))


5012
3508


In [8]:
print(torch.dot(labels_cat[1][0].T, output_cat[0]) == torch.LongTensor([20]))
print(torch.dot(labels_rnd[1][0].T, output_cat[0]) == torch.LongTensor([0]))

tensor([True])
tensor([True])


In [26]:
def load_dataset():
    data_path = 'cat-and-dog/training_set/'
    train_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=transform
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=64,
        num_workers=0,
        shuffle=True
    )
    return train_loader, train_dataset

#for batch_idx, (data, target) in enumerate(load_dataset()):
trainloader, traindataset = load_dataset()
classes = traindataset.classes
dict_classes = {'cats': output_cat, 'rnds': output_zero}
print(traindataset.classes)
print(dict_classes['cats'].shape)


['cats', 'rnds']
torch.Size([1, 1024])


In [27]:
def class_int2vector(lst_int_classes, classes, dict_classes):
    lst_aux = []
    for int_class in lst_int_classes:
        lst_aux.append(dict_classes[classes[int_class]])
        
    return cat_list(lst_aux)

def cats_cint2vec(lst_int_classes):
    return class_int2vector(lst_int_classes, classes=classes, dict_classes=dict_classes)


In [28]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        net.to(device)
        # get the inputs; data is a list of [inputs, labels]
        inputs, int_labels = data
        labels = cats_cint2vec(int_labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.to(device))
        loss = criterion(outputs, labels.float().to(device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')



RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 388 and 375 in dimension 2 at /opt/conda/conda-bld/pytorch_1565287142374/work/aten/src/TH/generic/THTensor.cpp:689

In [ ]:
acc = 0
for epoch in range(0):
    running_loss = 0.0
    for j, cat in enumerate(load_data_img(grp_dataset_nobkg['cats'])):
        #net.to(device)
        net
        optimizer.zero_grad()
        train, labels = create_minibatch(img=cat, label=output_cat, lbRand=output_zero, presetted=rnd_nocat[j%len(rnd_nocat)])
        #outputs = net(train.to(device))
        outputs = net(train)
        #print(outputs.shape)
        #print(labels.shape)
        #loss = criterion(outputs, labels.float().to(device))
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        acc += 1
        if acc % 1000 == 999:    # print every 10 mini-batches
            acc = 0
            print('[%d, %5d] loss: %.3f' %
            (epoch + 1, j + 1, running_loss / 1000))
            running_loss = 0.0
        


In [ ]:
def print_outputhat(idxs, output_y, output_yhat):
    v_idxs = {}
    for i in idxs:
        v_idxs[i] = float(output_yhat[0,i])

    print(torch.dot(output_y.reshape(1024).T, output_y.reshape(1024)))
    print(torch.dot(output_yhat.reshape(1024).T, output_yhat.reshape(1024)))
    print(output_yhat[0,idxs[0]-1])
    print(output_yhat[0,idxs[0]])
    print(v_idxs)
    print(output_yhat[0,0:100])
    
print_outputhat(rand_idxs, output_cat, out)


In [ ]:
def verify_bits(size, idxs, output_yhat, threshold=0.5):
    verify = torch.zeros((size))
    position = []
    print(verify.shape)
    for i in range(verify.shape[0]):
        if output_yhat[0,i] > threshold:
            position.append(i)
            verify[i] = 1
        else:
            verify[i] = 0

    idxs.sort()
    print(verify)
    print(position)
    print(idxs)
    print(torch.dot(verify.T, verify))
    
verify_bits(1024, rand_idxs, out, threshold=0.0001)


In [ ]:
test0 = transform(load_img(grp_dataset_nobkg['cats'][1]))
test0 = test0.unsqueeze(0)
imshow(torchvision.utils.make_grid(test0))


In [ ]:
out_t0 = net(test0.to(device))
print(out_t0)


In [ ]:
print_outputhat(rand_idxs, output_cat, out_t0)


In [ ]:
verify_bits(1024, rand_idxs, out_t0, threshold=0.0000014)


In [ ]:
def print_dot_out1_out2(out1, out2, size):
    print(torch.dot(out1.reshape(size).T, out2.reshape(size)))
    
print_dot_out1_out2(out, out_t0, 1024)

In [ ]:
test1 = transform(load_img(grp_dataset['cats'][0]))
test1 = test1.unsqueeze(0)
imshow(torchvision.utils.make_grid(test1))
out_t1 = net(test1.to(device))
print(out_t1)


In [ ]:
print_outputhat(rand_idxs, output_cat, out_t1)

In [ ]:
verify_bits(1024, rand_idxs, out, threshold=0.0000014)

In [ ]:
print_dot_out1_out2(out, out_t0, 1024)

In [ ]:
test_pl0 = transform(load_img(test_plane))
test_pl0 = test_pl0.unsqueeze(0)
imshow(torchvision.utils.make_grid(test_pl0))
out_plane0 = net(test_pl0.to(device))
print(out_plane0)


In [ ]:
print_outputhat(rand_idxs, output_cat, out_t1)

In [ ]:
verify_bits(1024, rand_idxs, out, threshold=0.0000013)

In [ ]:
print_dot_out1_out2(out, out_t0, 1024)


In [ ]:
#tconv1 = nn.Conv2d(3, 6, 5)
#tpool = nn.MaxPool2d(2, 2)
#tconv2 = nn.Conv2d(6, 16, 5)
#tfc1 = nn.Linear(16 * 5 * 5, 120)
#tfc2 = nn.Linear(120, 84)
#tfc3 = nn.Linear(84, 1024)
#transforms.Resize((300,400))
#resize = transforms.Resize((300,400))

for i in range(1):
    tx = tpool(F.relu(tconv1(train0)))
    print(tx.shape)
    tx = tpool(F.relu(tconv2(tx)))
    print(tx.shape)
    tx = tx.view(-1, 150544)
    print(tx.shape)
    tx = F.relu(tfc1(tx))
    print(tx.shape)
    tx = F.relu(tfc2(tx))
    print(tx.shape)
    tx = torch.sigmoid(tfc3(tx))
    print(tx.shape)

tx

In [ ]:
nr_epoch = 1
for epoch in range(nr_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, datapath in enumerate(grp_dataset['cats'], 0):
        # get the inputs; data is a list of [inputs, labels]
        #inputs, labels = data
        img=cv2.imread(datapath,cv2.IMREAD_COLOR)
        print(img.shape)
        print(transform(img).shape)
        trf_img = transform(img).reshape(1, img.shape[0], img.shape[1], img.shape[2])
        inputs, labels = trf_img.to(device), output_cat.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


In [ ]:
outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))
